In [63]:
from pymongo import MongoClient
from pymongo import errors
from pprint import pprint
import numpy as np
import re

In [64]:
def connect_database():
    client = MongoClient('localhost', 27017)
    return client

In [65]:
client = connect_database()

In [66]:
client.list_database_names()

['Kitchen_test',
 'Perfume_Database',
 'WorldDevelopmentIndicators',
 'admin',
 'config',
 'local']

In [67]:
perfume_database = client.Perfume_Database

In [68]:
pprint(perfume_database.list_collection_names())

['Cleared_Crawled_Perfumes',
 'Scraped_Perfumes',
 'Extra01',
 'Perfumes',
 'Extra02',
 'Fragrances',
 'Crawled_Perfumes']


In [69]:
extra02_collection = perfume_database.Extra02
fragrances_collection = perfume_database.Fragrances

# Compare the data sets

In [70]:
res = extra02_collection.aggregate([
  { "$sample": { "size": 5 } }
])

for i in res:
    pprint(i)

{'_id': ObjectId('65c3ecc62be6a0a6d64f6798'),
 'base_notes1': 'Amber',
 'base_notes2': 'Oakmoss',
 'base_notes3': 'Woody',
 'brand': 'S.TOI Fragrances',
 'concentration': 'EDP',
 'followers': 830,
 'instagram_link': 'https://www.instagram.com/stoiaroma/',
 'mid_notes1': 'Cedarwood',
 'mid_notes2': 'Honey',
 'mid_notes3': 'Spice',
 'price': '239,000.00',
 'price_per_ml': '4,780.00',
 'size': 50,
 'top_notes1': 'Calabrian Bergamot',
 'top_notes2': 'Grapefruit',
 'top_notes3': 'Hazelnut',
 'variant': 'Indigo',
 'variant_link': 'https://tokopedia.link/Yo9qMfn07xb',
 'year': 2023}
{'_id': ObjectId('65c3ecc62be6a0a6d64f67a7'),
 'base_notes1': 'Musk',
 'base_notes2': 'Cashmere',
 'base_notes3': 'Wood',
 'brand': 'The Pleasure',
 'concentration': 'EDP',
 'followers': 1048,
 'instagram_link': 'https://www.instagram.com/thepleasure.id/',
 'mid_notes1': 'Vanilla',
 'mid_notes2': 'Floral',
 'mid_notes3': 'Almond',
 'price': '195,000.00',
 'price_per_ml': '3,900.00',
 'size': 50,
 'top_notes1': 'Co

In [71]:
pprint(fragrances_collection.find_one())

{'_id': ObjectId('65a54ac72b32912eefb9ea54'),
 'base notes': ['Civet', 'Resins', 'Musk'],
 'company': 'Bogue',
 'description': 'Maai by Bogue is a Floral Woody Musk fragrance for women and '
                'men. Maai was launched in 2014. The nose behind this '
                'fragrance is Antonio Gardoni. Top notes are Aldehydes and '
                'Tuberose; middle notes are Jasmine and Rose; base notes are '
                'Civet, Resins and Musk. .',
 'gender': 'for women and men',
 'gender_vote': {'average': 0.2, 'votes_count': 130},
 'image': 'https://fimgs.net/mdimg/perfume/375x500.26681.jpg',
 'longevity': {'eternal': 69,
               'long lasting': 140,
               'moderate': 28,
               'very weak': 23,
               'weak': 9},
 'main accords': {'aldehydic': 71.419,
                  'amber': 70.8567,
                  'animalic': 80.9679,
                  'balsamic': 44.2436,
                  'fresh': 57.3066,
                  'musky': 100.0,
        

# gather notes in their own array

In [72]:
document_into_array = {
        '$project': {
            'fields': {
                '$objectToArray': '$$ROOT'
            }
        }
    }

In [73]:
def filter_fields(regex_filter):
    return {
        '$project': {
            'fieldsArray': {
                '$filter': {
                    'input': '$fields', 
                    'as': 'item', 
                    'cond': {
                        '$regexMatch': {
                            'input': '$$item.k', 
                            'regex': regex_filter
                        }
                    }
                }
            }
        }
    }

In [74]:
def accumulate_field_values(field_name):
    return {
        '$project': {
            field_name: {
                '$map': {
                    'input': '$fieldsArray', 
                    'as': 'item', 
                    'in': '$$item.v'
                }
            }
        }
    }

In [75]:
accumulate_top_notes_pipeline = [
    document_into_array,
    filter_fields("^top"),
    accumulate_field_values("top notes")
]

In [ ]:
add_other_fields = {}

In [76]:
result = extra02_collection.aggregate(accumulate_top_notes_pipeline)

In [77]:
pprint(result.next())

{'_id': ObjectId('65c3ecc62be6a0a6d64f6747'),
 'top notes': ['Fresh Watery Accord', 'Rose', 'Pink Pepper']}
